In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from ase.io import read

# new_path = '/capstor/scratch/cscs/ajayaraj/qtpyt-tests/pentacene'
# os.chdir(new_path)

In [7]:
atoms = read("./dft/device/scatt_sorted.xyz")

z_threshold = 8  # Separate leads vs bridge (in z)
x_coords = atoms.positions[:, 0]
z_coords = atoms.positions[:, 2]

x_min, x_max = np.min(x_coords), np.max(x_coords)
x_threshold = (x_max + x_min) * 0.5  # Midpoint in x

symbols = atoms.get_chemical_symbols()
total_atoms = len(atoms)

num_spherical_harmonics = {"C": 9, "H": 4}

left_lead_indices = []
right_lead_indices = []
bridge_indices = []

for i, (x, z) in enumerate(zip(x_coords, z_coords)):
    if z < z_threshold:
        if x < x_threshold:
            left_lead_indices.append(i)
        else:
            right_lead_indices.append(i)
    else:
        bridge_indices.append(i)

left_lead_indices = np.array(left_lead_indices)
right_lead_indices = np.array(right_lead_indices)
bridge_indices = np.array(bridge_indices)

# Debug prints
print("Total atoms:", total_atoms)
print("Left lead atoms:", len(left_lead_indices))
print("Right lead atoms:", len(right_lead_indices))
print("Bridge atoms:", len(bridge_indices))

# Compute total spherical harmonics per region
total_sph_left_lead = sum(num_spherical_harmonics[symbols[i]] for i in left_lead_indices)
total_sph_bridge = sum(num_spherical_harmonics[symbols[i]] for i in bridge_indices)
total_sph_right_lead = sum(num_spherical_harmonics[symbols[i]] for i in right_lead_indices)

nodes = [
    0,
    total_sph_left_lead,
    total_sph_left_lead + total_sph_bridge,
    total_sph_left_lead + total_sph_bridge + total_sph_right_lead
]

print("Number of orbitals in left lead =", nodes[1] - nodes[0])
print("Number of orbitals in bridge =", nodes[2] - nodes[1])
print("Number of orbitals in right lead =", nodes[3] - nodes[2])

np.save("./output/device/nodes.npy", nodes)


Total atoms: 114
Left lead atoms: 40
Right lead atoms: 38
Bridge atoms: 36
Number of orbitals in left lead = 290
Number of orbitals in bridge = 254
Number of orbitals in right lead = 272
